# Orbital Mechanics: Kepler's Laws and Planetary Motion

## Introduction

Johannes Kepler's three laws of planetary motion, published between 1609 and 1619, represent one of the foundational achievements in celestial mechanics. These empirical laws describe the motion of planets around the Sun and were later derived from Newton's law of universal gravitation.

## Kepler's Three Laws

### First Law (Law of Ellipses)

The orbit of every planet is an ellipse with the Sun at one of the two foci. The equation of an ellipse in polar coordinates with the focus at the origin is:

$$r(\theta) = \frac{a(1 - e^2)}{1 + e\cos\theta}$$

where:
- $r$ is the radial distance from the focus (Sun)
- $\theta$ is the true anomaly (angle from perihelion)
- $a$ is the semi-major axis
- $e$ is the orbital eccentricity ($0 \leq e < 1$ for ellipses)

### Second Law (Law of Equal Areas)

A line segment joining a planet and the Sun sweeps out equal areas during equal intervals of time. Mathematically:

$$\frac{dA}{dt} = \frac{1}{2}r^2\frac{d\theta}{dt} = \frac{L}{2m} = \text{constant}$$

where $L$ is the orbital angular momentum and $m$ is the planet's mass.

### Third Law (Harmonic Law)

The square of the orbital period of a planet is directly proportional to the cube of the semi-major axis of its orbit:

$$T^2 = \frac{4\pi^2}{GM}a^3$$

where:
- $T$ is the orbital period
- $G$ is the gravitational constant
- $M$ is the mass of the central body (Sun)

## Derivation from Newton's Laws

The gravitational force between the Sun (mass $M$) and planet (mass $m$) is:

$$\vec{F} = -\frac{GMm}{r^2}\hat{r}$$

This central force leads to conservation of angular momentum $\vec{L} = \vec{r} \times \vec{p}$, which confines motion to a plane and implies Kepler's second law.

The orbit equation can be derived by solving the equation of motion in polar coordinates:

$$\frac{d^2u}{d\theta^2} + u = \frac{GM}{h^2}$$

where $u = 1/r$ and $h = L/m$ is the specific angular momentum.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from scipy.optimize import brentq

# Set up matplotlib for publication-quality figures
plt.rcParams['figure.figsize'] = [12, 10]
plt.rcParams['font.size'] = 11
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 14

## Numerical Simulation of Orbital Motion

We will simulate planetary orbits by numerically integrating the equations of motion. The state vector is $\vec{y} = (x, y, v_x, v_y)$, and the equations of motion are:

$$\frac{d\vec{r}}{dt} = \vec{v}, \quad \frac{d\vec{v}}{dt} = -\frac{GM}{r^3}\vec{r}$$

In [ ]:
# Physical constants (using normalized units where GM = 1)
GM = 1.0  # Gravitational parameter (normalized)

def orbital_equations(state, t, GM):
    """
    Equations of motion for two-body problem.
    
    Parameters:
    -----------
    state : array
        [x, y, vx, vy] - position and velocity
    t : float
        Time (unused for autonomous system)
    GM : float
        Gravitational parameter
    
    Returns:
    --------
    derivatives : array
        [vx, vy, ax, ay]
    """
    x, y, vx, vy = state
    r = np.sqrt(x**2 + y**2)
    
    # Accelerations
    ax = -GM * x / r**3
    ay = -GM * y / r**3
    
    return [vx, vy, ax, ay]

def compute_orbital_elements(state, GM):
    """
    Compute orbital elements from state vector.
    
    Returns:
    --------
    a : semi-major axis
    e : eccentricity
    """
    x, y, vx, vy = state
    r = np.sqrt(x**2 + y**2)
    v = np.sqrt(vx**2 + vy**2)
    
    # Specific orbital energy
    energy = 0.5 * v**2 - GM / r
    
    # Semi-major axis
    a = -GM / (2 * energy)
    
    # Specific angular momentum
    h = x * vy - y * vx
    
    # Eccentricity
    e = np.sqrt(1 + 2 * energy * h**2 / GM**2)
    
    return a, e

## Demonstration of Kepler's First Law

We simulate orbits with different eccentricities to demonstrate that bound orbits are ellipses.

In [ ]:
def generate_initial_conditions(a, e, GM):
    """
    Generate initial conditions at perihelion for given orbital elements.
    
    At perihelion: r = a(1-e), velocity is perpendicular to radius
    """
    r_perihelion = a * (1 - e)
    
    # Velocity at perihelion from vis-viva equation: v^2 = GM(2/r - 1/a)
    v_perihelion = np.sqrt(GM * (2/r_perihelion - 1/a))
    
    # Initial state: at perihelion on positive x-axis, velocity in +y direction
    return [r_perihelion, 0, 0, v_perihelion]

# Create figure for multiple orbit visualizations
fig = plt.figure(figsize=(14, 12))

# --- Panel 1: Orbits with different eccentricities ---
ax1 = fig.add_subplot(2, 2, 1)

eccentricities = [0.0, 0.3, 0.6, 0.8]
colors = ['#2E86AB', '#A23B72', '#F18F01', '#C73E1D']
a = 1.0  # Semi-major axis

for e, color in zip(eccentricities, colors):
    # Initial conditions
    state0 = generate_initial_conditions(a, e, GM)
    
    # Orbital period from Kepler's third law
    T = 2 * np.pi * np.sqrt(a**3 / GM)
    t = np.linspace(0, T, 1000)
    
    # Integrate orbit
    solution = odeint(orbital_equations, state0, t, args=(GM,))
    x, y = solution[:, 0], solution[:, 1]
    
    ax1.plot(x, y, color=color, linewidth=1.5, label=f'e = {e}')

# Plot Sun at focus
ax1.plot(0, 0, 'yo', markersize=15, markeredgecolor='orange', label='Sun')

ax1.set_xlabel('x (AU)')
ax1.set_ylabel('y (AU)')
ax1.set_title("Kepler's First Law: Elliptical Orbits")
ax1.legend(loc='upper right')
ax1.set_aspect('equal')
ax1.grid(True, alpha=0.3)

# --- Panel 2: Kepler's Second Law - Equal Areas ---
ax2 = fig.add_subplot(2, 2, 2)

# Use moderately eccentric orbit
e = 0.6
state0 = generate_initial_conditions(a, e, GM)
T = 2 * np.pi * np.sqrt(a**3 / GM)
t = np.linspace(0, T, 1000)
solution = odeint(orbital_equations, state0, t, args=(GM,))
x, y = solution[:, 0], solution[:, 1]

# Plot full orbit
ax2.plot(x, y, 'k-', linewidth=1, alpha=0.5)

# Highlight equal-time segments
n_segments = 8
segment_indices = np.linspace(0, len(t)-1, n_segments+1, dtype=int)
colors_segments = plt.cm.viridis(np.linspace(0, 1, n_segments))

for i in range(n_segments):
    idx_start, idx_end = segment_indices[i], segment_indices[i+1]
    
    # Create filled wedge
    wedge_x = np.concatenate([[0], x[idx_start:idx_end+1], [0]])
    wedge_y = np.concatenate([[0], y[idx_start:idx_end+1], [0]])
    ax2.fill(wedge_x, wedge_y, color=colors_segments[i], alpha=0.6)

ax2.plot(0, 0, 'yo', markersize=12, markeredgecolor='orange')
ax2.set_xlabel('x (AU)')
ax2.set_ylabel('y (AU)')
ax2.set_title("Kepler's Second Law: Equal Areas in Equal Times")
ax2.set_aspect('equal')
ax2.grid(True, alpha=0.3)

# --- Panel 3: Kepler's Third Law ---
ax3 = fig.add_subplot(2, 2, 3)

# Simulate orbits with different semi-major axes
semi_major_axes = np.array([0.5, 0.75, 1.0, 1.5, 2.0, 3.0])
measured_periods = []

for a_test in semi_major_axes:
    e_test = 0.3
    state0 = generate_initial_conditions(a_test, e_test, GM)
    
    # Theoretical period
    T_theory = 2 * np.pi * np.sqrt(a_test**3 / GM)
    t = np.linspace(0, 2*T_theory, 2000)
    
    solution = odeint(orbital_equations, state0, t, args=(GM,))
    
    # Measure period by finding when orbit completes (y crosses zero going positive)
    y_vals = solution[:, 1]
    vy_vals = solution[:, 3]
    
    # Find first complete orbit
    for j in range(1, len(y_vals)):
        if y_vals[j-1] < 0 and y_vals[j] >= 0 and vy_vals[j] > 0:
            # Linear interpolation for more accurate crossing time
            T_measured = t[j-1] + (t[j] - t[j-1]) * (-y_vals[j-1]) / (y_vals[j] - y_vals[j-1])
            measured_periods.append(T_measured)
            break

measured_periods = np.array(measured_periods)

# Plot T^2 vs a^3
ax3.scatter(semi_major_axes**3, measured_periods**2, s=100, c='#2E86AB', 
            edgecolors='black', linewidth=1, label='Simulated', zorder=5)

# Theoretical line
a_theory = np.linspace(0, 3.5**3, 100)
T2_theory = (4 * np.pi**2 / GM) * a_theory
ax3.plot(a_theory, T2_theory, 'r-', linewidth=2, label='Theory: $T^2 = (4\pi^2/GM)a^3$')

ax3.set_xlabel('$a^3$ (AU$^3$)')
ax3.set_ylabel('$T^2$ (years$^2$)')
ax3.set_title("Kepler's Third Law: $T^2 \propto a^3$")
ax3.legend()
ax3.grid(True, alpha=0.3)

# --- Panel 4: Orbital Velocity ---
ax4 = fig.add_subplot(2, 2, 4)

# Plot velocity variation for elliptical orbit
e = 0.6
state0 = generate_initial_conditions(a, e, GM)
T = 2 * np.pi * np.sqrt(a**3 / GM)
t = np.linspace(0, T, 1000)
solution = odeint(orbital_equations, state0, t, args=(GM,))

# Calculate radial distance and velocity magnitude
x, y, vx, vy = solution.T
r = np.sqrt(x**2 + y**2)
v = np.sqrt(vx**2 + vy**2)

# True anomaly
theta = np.arctan2(y, x)
theta = np.unwrap(theta)  # Make continuous

# Plot velocity vs true anomaly
ax4.plot(np.degrees(theta), v, 'b-', linewidth=2, label='Orbital velocity')

# Mark perihelion and aphelion
ax4.axvline(x=0, color='g', linestyle='--', alpha=0.7, label='Perihelion')
ax4.axvline(x=180, color='r', linestyle='--', alpha=0.7, label='Aphelion')

# Vis-viva equation prediction
theta_theory = np.linspace(0, 360, 1000)
r_theory = a * (1 - e**2) / (1 + e * np.cos(np.radians(theta_theory)))
v_theory = np.sqrt(GM * (2/r_theory - 1/a))
ax4.plot(theta_theory, v_theory, 'k--', linewidth=1, alpha=0.5, label='Vis-viva equation')

ax4.set_xlabel('True Anomaly (degrees)')
ax4.set_ylabel('Orbital Velocity (AU/year)')
ax4.set_title('Velocity Variation in Elliptical Orbit (e=0.6)')
ax4.legend(loc='upper right')
ax4.grid(True, alpha=0.3)
ax4.set_xlim(0, 360)

plt.tight_layout()
plt.savefig('plot.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nVerification of Kepler's Third Law:")
print("="*50)
print(f"{'a (AU)':<10} {'T² (measured)':<15} {'T² (theory)':<15} {'Error %':<10}")
print("-"*50)
for a_val, T_meas in zip(semi_major_axes, measured_periods):
    T2_meas = T_meas**2
    T2_theory = (4 * np.pi**2 / GM) * a_val**3
    error = abs(T2_meas - T2_theory) / T2_theory * 100
    print(f"{a_val:<10.2f} {T2_meas:<15.4f} {T2_theory:<15.4f} {error:<10.4f}")

## Analysis and Discussion

### Results Summary

The numerical simulations confirm all three of Kepler's laws:

1. **First Law**: The orbits traced by numerical integration are closed ellipses with the central body at one focus. As eccentricity increases, the orbit becomes more elongated.

2. **Second Law**: The colored wedges in the second panel all have equal areas despite covering different arc lengths. Near perihelion (closest approach), the planet moves faster, sweeping through a larger angle in the same time as at aphelion.

3. **Third Law**: The plot of $T^2$ versus $a^3$ shows a perfect linear relationship, confirming $T^2 = \frac{4\pi^2}{GM}a^3$.

### Physical Implications

The velocity plot demonstrates an important consequence of energy conservation. From the vis-viva equation:

$$v = \sqrt{GM\left(\frac{2}{r} - \frac{1}{a}\right)}$$

At perihelion ($r = a(1-e)$), velocity is maximum, while at aphelion ($r = a(1+e)$), velocity is minimum. This velocity ratio is:

$$\frac{v_{\text{perihelion}}}{v_{\text{aphelion}}} = \frac{1+e}{1-e}$$

### Applications

Kepler's laws are fundamental to:
- Spacecraft trajectory design and orbital maneuvers
- Exoplanet detection via radial velocity and transit methods
- Binary star system analysis
- Satellite orbit determination